# C-SWM 

In [1]:
#Libs
import Base: iterate, length, GC
using HDF5
using Knet
using Statistics: mean,std
using Random
using LinearAlgebra

#Datatype
atype=KnetArray{Float32}

#Includes
include("datasets.jl")
include("cswm.jl")

## Model Definition

In [2]:
#Params
input_ch = 3
hidden_dim = 512
num_objects = 5
embedding_dim = 2
action_dim = 2
sigma = 0.5
hinge = 1.0

1.0

In [3]:
Knet.seed!(42)

MersenneTwister(UInt32[0x0000002a], Random.DSFMT.DSFMT_state(Int32[964434469, 1073036706, 1860149520, 1073503458, 1687169063, 1073083486, -399267803, 1072983952, -909620556, 1072836235  …  -293054293, 1073002412, -1300127419, 1073642642, 1917177374, -666058738, -337596527, 1830741494, 382, 0]), [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], UInt128[0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000  …  0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x0000000000

In [4]:
model = initContrastiveSWMLarge(input_ch, hidden_dim, num_objects, embedding_dim, action_dim, sigma, hinge)

ContrastiveSWM(EncoderCNNLarge(Any[P(KnetArray{Float32,4}(3,3,3,32)), P(KnetArray{Float32,4}(3,3,32,32)), P(KnetArray{Float32,4}(3,3,32,32)), P(KnetArray{Float32,4}(3,3,32,5))], Any[P(KnetArray{Float32,4}(1,1,32,1)), P(KnetArray{Float32,4}(1,1,32,1)), P(KnetArray{Float32,4}(1,1,32,1)), P(KnetArray{Float32,4}(1,1,5,1))], Knet.sigm, NNlib.relu), EncoderMLP(Param{KnetArray{Float32,2}}[P(KnetArray{Float32,2}(512,2500)), P(KnetArray{Float32,2}(512,512)), P(KnetArray{Float32,2}(2,512))], Param{KnetArray{Float32,1}}[P(KnetArray{Float32,1}(512)), P(KnetArray{Float32,1}(512)), P(KnetArray{Float32,1}(2))], NNlib.relu), TransitionGNN(EdgeMLP(Param{KnetArray{Float32,2}}[P(KnetArray{Float32,2}(512,4)), P(KnetArray{Float32,2}(512,512)), P(KnetArray{Float32,2}(512,512))], Param{KnetArray{Float32,1}}[P(KnetArray{Float32,1}(512)), P(KnetArray{Float32,1}(512)), P(KnetArray{Float32,1}(512))], NNlib.relu), NodeMLP(Param{KnetArray{Float32,2}}[P(KnetArray{Float32,2}(512,516)), P(KnetArray{Float32,2}(512,512

## Training Part

In [5]:
TRAIN_DATASET_PATH = "/home/cagan/dev/datasets/moving_mnist/movingmnist_trn_moveone.h5"
TRAIN_BATCH_SIZE = 128
dtrn = buildMovingMNISTVelocityDataset(TRAIN_DATASET_PATH, true, TRAIN_BATCH_SIZE);

Dataset loaded. Building dataset indexing...
Done.


In [ ]:
obs, action, next_obs = first(dtrn)

In [ ]:
action

In [6]:
function initopt!(model::ContrastiveSWM)
    
    for par in params(model)
        par.opt = Adam(;lr=0.0005, gclip=0, beta1=0.9, beta2=0.999, eps=1e-8)
        println(par)
    end
end;
initopt!(model)

P(KnetArray{Float32,4}(3,3,3,32))
P(KnetArray{Float32,4}(3,3,32,32))
P(KnetArray{Float32,4}(3,3,32,32))
P(KnetArray{Float32,4}(3,3,32,5))
P(KnetArray{Float32,4}(1,1,32,1))
P(KnetArray{Float32,4}(1,1,32,1))
P(KnetArray{Float32,4}(1,1,32,1))
P(KnetArray{Float32,4}(1,1,5,1))
P(KnetArray{Float32,2}(512,2500))
P(KnetArray{Float32,2}(512,512))
P(KnetArray{Float32,2}(2,512))
P(KnetArray{Float32,1}(512))
P(KnetArray{Float32,1}(512))
P(KnetArray{Float32,1}(2))
P(KnetArray{Float32,2}(512,4))
P(KnetArray{Float32,2}(512,512))
P(KnetArray{Float32,2}(512,512))
P(KnetArray{Float32,1}(512))
P(KnetArray{Float32,1}(512))
P(KnetArray{Float32,1}(512))
P(KnetArray{Float32,2}(512,516))
P(KnetArray{Float32,2}(512,512))
P(KnetArray{Float32,2}(2,512))
P(KnetArray{Float32,1}(512))
P(KnetArray{Float32,1}(512))
P(KnetArray{Float32,1}(2))


In [7]:
#Verbose after x batches
VERBOSE =  10

#Define number of epochs
NUM_EPOCHS = 100

println("Starting training...")

for i in 1:NUM_EPOCHS
    
    avg_loss = 0.0
    it = 0
    for  (k, (obs, action, next_obs)) in enumerate(dtrn)

        #Train by using contrastive loss
        J = @diff model(obs,action,next_obs)
        
        for par in params(model)
            g = grad(J, par)
            update!(value(par), g, par.opt)
        end
        
        batch_size = size(obs,4)

        if k % VERBOSE == 0
            
            println("Epoch: ", i , ", Iter: " , k*batch_size, "/", dtrn.num_steps, ", Loss: ", value(J))

        end
        
        avg_loss += value(J)
        it = k
        
    end
    
    avg_loss /= it
    
    println("Avg loss: " , avg_loss)
end

#dtrn = nothing
#Knet.gc()
#GC.gc()

Starting training...
Epoch: 1, Iter: 1280/20000, Loss: 1.0721711
Epoch: 1, Iter: 2560/20000, Loss: 1.0079288
Epoch: 1, Iter: 3840/20000, Loss: 0.9998389
Epoch: 1, Iter: 5120/20000, Loss: 1.000047
Epoch: 1, Iter: 6400/20000, Loss: 0.9995246
Epoch: 1, Iter: 7680/20000, Loss: 0.9963907
Epoch: 1, Iter: 8960/20000, Loss: 0.76921374
Epoch: 1, Iter: 10240/20000, Loss: 0.63122284
Epoch: 1, Iter: 11520/20000, Loss: 0.30674505
Epoch: 1, Iter: 12800/20000, Loss: 0.28273505
Epoch: 1, Iter: 14080/20000, Loss: 0.24841025
Epoch: 1, Iter: 15360/20000, Loss: 0.2383742
Epoch: 1, Iter: 16640/20000, Loss: 0.24772957
Epoch: 1, Iter: 17920/20000, Loss: 0.2067228
Epoch: 1, Iter: 19200/20000, Loss: 0.19443269
Avg loss: 0.6644344270611421
Epoch: 2, Iter: 1280/20000, Loss: 0.20733115
Epoch: 2, Iter: 2560/20000, Loss: 0.19151682
Epoch: 2, Iter: 3840/20000, Loss: 0.18177727
Epoch: 2, Iter: 5120/20000, Loss: 0.13254283
Epoch: 2, Iter: 6400/20000, Loss: 0.14573814
Epoch: 2, Iter: 7680/20000, Loss: 0.15681615
Epoch:

In [ ]:
model(obs)

In [ ]:
dtrn = nothing
Knet.gc()
GC.gc()

In [11]:
Knet.save("model_mvmnist_moveone.jld2", "model", model)

## Evaluation

In [ ]:
model = Knet.load("model_mvmnist_slow.jld2", "model", model)

In [13]:
#Params
EVAL_DATASET_PATH = "/home/cagan/dev/datasets/moving_mnist/movingmnist_tst_moveone.h5"
EVAL_BATCH_SIZE = 100

100

In [14]:
dtst = buildMovingMNISTMultiStepPathDataset(EVAL_DATASET_PATH, EVAL_BATCH_SIZE,1);

In [15]:
pred_states = Any[]
next_states = Any[]

num_samples = dtst.dataset_size

for  (k, (obs, action, next_obs)) in enumerate(dtst)
    
    if k % 10 == 0
        
        println("Processed ", k ," batches")
        
    end
    #Obs => (50,50,3,100)
    #Next obs => (50,50,3,100)
    
    pred_state = Array{Float32}(model(obs,action[:,:,1]) + model(obs))
    next_state = Array{Float32}(model(next_obs))
    
    #Pred-state => (2,5,100)
    #Next state => (2,5,100)
    #println(pred_state)
    #println(next_state)
    
    push!(pred_states, pred_state)
    push!(next_states, next_state)
    
end

#Pred state cat => [2,5,10000]
#Next state cat => [2,5,10000]
pred_states = cat(pred_states...,dims=3)
next_states = cat(next_states...,dims=3)
    
#Flatten object/feature dimensions
pred_states = mat(pred_states)  #[10,10000]
next_states = mat(next_states)  #[10,10000]

#Calculate pairwise distances
sizes_1 = (size(pred_states)...,1)
sizes_2 = (sizes_1[1], sizes_1[3], sizes_1[2])

pred_states = reshape(pred_states, sizes_1)
next_states = reshape(next_states, sizes_2)
pred_states = repeat(pred_states, outer=[1,1,1000])
next_states = repeat(next_states, outer=[1,1000,1])

pairwise_distance_matrix = sum((pred_states - next_states).^2, dims=1)[1,:,:]

#Augment pairwise distance matrix
diag_elements = diag(pairwise_distance_matrix)
pairwise_distance_matrix = hcat(diag_elements, pairwise_distance_matrix)


labels = ones(num_samples)
hits_at_1 = 0

indices = []

for i=1:1000
    
    row = pairwise_distance_matrix[i,:]
    ind = sortperm(row)
    
    push!(indices, ind)

end

indices = vcat(indices'...)

Processed 10 batches


1000×1001 Array{Int64,2}:
   1     2   637  273  186  317  823  …  137  387  818  546  209  149  542
   1     3   771  710  213  522  421     391  475  699  353  195  584  803
 520     1     4  997  571  433   13     264  932  603  634  127  142  779
   1     5   837  803  901  117  702     110   63  546  101  176  149  779
   1     6   833  243  157  223  974     656  114  634  931   72  531  142
 438     1     7  546  503  846  137  …  890  195  699  333  475  624  637
   1     8   833  157  207  751  872     656  142  127  931  380  468  446
   1     9    80  736  511  724  624     387  315  209  542  541  478  149
 541     1    10  441  396  340  697     552  724  679  890  195  637  624
   1    11   511  784   28  397  772     818  503  478  209  546  387  149
   1    12   609  176  878  196  116  …  761  999  624  837  507  630  803
 353     1    13  520  445  799  103     387  672  546   74   63  818  142
   1    14   350  959  675  238  684     531  149  101  962  142   76  387

In [16]:
num_matches = sum(labels .== indices[:,1])
println("Hits @ 1: ", num_matches/num_samples)

Hits @ 1: 0.698


In [17]:
num_matches = sum(labels .== indices[:,1:5])
println("Hits @ 5: ", num_matches/num_samples)

Hits @ 5: 0.991


In [18]:
mxval, mxindx = findmax(indices .== labels,dims=2)
ranks = [ i[2] for i in mxindx ]
reciprocal_ranks = 1 ./ranks
rr_sum = sum(reciprocal_ranks)
println("MRR: ", rr_sum/num_samples)

MRR: 0.8261574786324787
